In [1]:
# --- Local Setup ---
import os
import sys
from pathlib import Path

# Add repository root to path
# Assuming notebook is in notebooks/ and repo root is one level up
repo_root = Path("..").resolve()
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

# Change working directory to repo root so config files are found correctly
os.chdir(repo_root)

In [8]:
# setup Environment & Config
import yaml
from src.config import load_settings

# Define local data paths
# User specified ./data/ contains data, models, results, runs, yoloConfig
# We assume this folder is in the repository root
catnip_data_root = repo_root / "catnip-data"
data_dir = catnip_data_root / "data"

# Override data path to point to local synced folder
os.environ["CATNIP_PATHS_DATA"] = str(data_dir)

settings = load_settings()
print(f"Data Root: {settings.paths.data}")
print(f"Manga Dir: {settings.paths.manga_dir}")
print(f"Annotations Dir: {settings.paths.annotations_dir}")



Data Root: data
Manga Dir: catnip-data\data\manga
Annotations Dir: data\annotations


In [ ]:
# prepare Dataset for YOLO
# my dataset uses 'manga' for images and 'annotations' for labels
# YOLO expects 'images' for images and 'labels' for labels
# we'll create symlinks to satisfy YOLO's expectations:

data_root = settings.paths.data
images_link = data_root / "images"
labels_link = data_root / "labels"

def safe_symlink(target, link_name):
    target = Path(target)
    link_name = Path(link_name)
    if not link_name.exists():
        try:
            os.symlink(target, link_name)
            print(f"Created symlink: {link_name} -> {target}")
        except OSError as e:
            print(f"Failed to create symlink {link_name} -> {target}: {e}")
            print("On Windows, you may need to run VS Code as Administrator or enable Developer Mode.")

if data_root.exists():
    safe_symlink(settings.paths.manga_dir, images_link)
    safe_symlink(settings.paths.annotations_dir, labels_link)
else:
    print(f"Warning: Data root {data_root} does not exist. Please ensure catnip-data is synced.")

In [ ]:
# convert label studio json to yolo format
import json

def convert_label_studio_to_yolo(json_path, output_dir, class_map):
    """converts label studio json export to yolo format txt files."""
    with open(json_path, 'r') as f:
        tasks = json.load(f)
        
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    count = 0
    for task in tasks:
        url = task['data']['url']
        if "manga/" in url:
            rel_path = url.split("manga/")[-1]
        else:
            continue
            
        image_rel_path = Path(rel_path)
        label_rel_path = image_rel_path.with_suffix('.txt')
        label_full_path = output_dir / label_rel_path
        
        label_full_path.parent.mkdir(parents=True, exist_ok=True)
        
        yolo_lines = []
        if 'annotations' in task:
            for annotation in task['annotations']:
                if 'result' in annotation:
                    for result in annotation['result']:
                        if 'type' in result and result['type'] == 'rectanglelabels':
                            value = result['value']
                            labels = value.get('rectanglelabels', [])
                            if not labels: continue
                            
                            label_name = labels[0]
                            if label_name not in class_map: continue
                                
                            class_id = class_map[label_name]
                            x, y, w, h = value['x'], value['y'], value['width'], value['height']
                            
                            x_center = (x + w / 2) / 100.0
                            y_center = (y + h / 2) / 100.0
                            w_norm = w / 100.0
                            h_norm = h / 100.0
                            
                            yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")
        
        with open(label_full_path, 'w') as f:
            f.write('\n'.join(yolo_lines))
        count += 1
    print(f"processed {count} tasks. labels saved to {output_dir}")


# Update path to local export
# Assuming exports are in the repo's data folder
json_file = repo_root / "data" / "ls-exports" / "251226v1.json"

if json_file.exists():
    convert_label_studio_to_yolo(json_file, settings.paths.annotations_dir, { "izutsumi": 0, "izutsumi_face": 1 })
else:
    print(f"Label Studio export not found at {json_file}")

In [ ]:
# generate or load training list based on available labels
# since we only labeled a subset of the images, we need to tell yolo exactly which images to use.

# define config directory mapping to ./catnip-data/yoloConfig/
config_dir = catnip_data_root / "yoloConfig"
config_dir.mkdir(parents=True, exist_ok=True)

# determine filename from input json if available
if 'json_file' in locals() and json_file.exists():
    train_list_name = f"{json_file.stem}_train.txt"
else:
    train_list_name = "train.txt"
    print("json_file not found or defined, defaulting to 'train.txt'")

train_list_path = config_dir / train_list_name

# Set to True to force regeneration of the list (useful if paths changed)
force_regenerate = True 

if train_list_path.exists() and not force_regenerate:
    print(f"found existing training list: {train_list_path}")
    with open(train_list_path, 'r') as f:
        lines = f.readlines()
    print(f"loaded {len(lines)} images from existing list.")
else:
    print(f"generating new training list: {train_list_path}")
    
    image_files = list(images_link.rglob("*.jpg")) + list(images_link.rglob("*.png")) + list(images_link.rglob("*.jpeg"))
    print(f"found {len(image_files)} total images in 'manga' directory.")

    labeled_images = []
    unlabeled_count = 0

    for img_path in image_files:
        # construct expected label path
        try:
            rel_path = img_path.relative_to(images_link)
            label_rel_path = rel_path.with_suffix(".txt")
            label_path = labels_link / label_rel_path
            
            if label_path.exists():
                # use absolute path to avoid ambiguity
                labeled_images.append(str(img_path.absolute()))
            else:
                unlabeled_count += 1
        except ValueError:
            continue

    # write train list
    with open(train_list_path, "w") as f:
        f.write("\n".join(labeled_images))

    print(f"generated {train_list_path}")
    print(f"   - labeled images (subset): {len(labeled_images)}")
    print(f"   - unlabeled images (skipped): {unlabeled_count}")

    if len(labeled_images) == 0:
        print("warning: no labeled images found.")

In [ ]:
# create dataset.yaml
# define the dataset configuration for YOLO

dataset_yaml = {
    'path': str(data_root),
    'train': str(train_list_path),  # point to the generated/loaded list
    'val': str(train_list_path),    # using same set for val for now
    
    # class names
    'names': {
        0: 'izutsumi',
        1: 'izutsumi_face'
    }
}

yaml_path = Path("dataset.yaml")
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f)

print(f"created {yaml_path}")
!cat {yaml_path}

In [ ]:
from ultralytics import YOLO

# load model
model = YOLO("yolo11s.pt") 

# train model
# project points to where runs are saved
project_dir = catnip_data_root / "runs"

results = model.train(
    data=str(yaml_path),
    epochs=100,
    imgsz=640,
    project=str(project_dir),
    name="izutsumi_v1",
    device="cuda", # "mps", "cuda", "cpu"
    cache=True, 
    exist_ok=True
)

In [ ]:
# save Model to Bucket Models Directory
import shutil

# the best model is saved in project_dir/name/weights/best.pt
best_model_path = project_dir / "izutsumi_v1" / "weights" / "best.pt"
target_model_dir = catnip_data_root / "models"
target_model_path = target_model_dir / "yolo11_izutsumi_trained.pt"

if best_model_path.exists():
    target_model_dir.mkdir(parents=True, exist_ok=True)
    shutil.copy(best_model_path, target_model_path)
    print(f"Model saved to {target_model_path}")
else:
    print("Training might have failed, best.pt not found.")

In [ ]:
# evaluate
metrics = model.val()
print(metrics) 

In [5]:
from ultralytics import YOLO
project_dir = catnip_data_root / "runs"

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\rifusaki\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# run inference on the entire dataset
import cv2
import os

# load the trained model
model_path = project_dir / "izutsumi_v1" / "weights" / "best.pt"
model = YOLO(model_path)

# define output directory mapping to ./catnip-data/results/
results_dir = catnip_data_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

# Use the actual manga directory directly
inference_source = settings.paths.manga_dir

# Create a specific output directory for flattened results
output_dir = results_dir / "inference"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"running inference on {inference_source}...")
print(f"saving flattened results to {output_dir}")

# run prediction recursively
# source=str(inference_source) will scan recursively if it's a directory
results = model.predict(
    source=str(inference_source) + "/**/*.*",  # recursive glob pattern
    project=str(results_dir),
    name="inference",
    save=False,      # Disable auto-save to handle manually
    save_txt=False,  # Disable auto-txt to handle manually
    conf=0.25,      # confidence threshold
    stream=True     # use generator to handle large datasets
)

# process results generator to trigger predictions and save manually
for r in results:
    # Calculate flattened filename
    original_path = Path(r.path)
    try:
        rel_path = original_path.relative_to(inference_source)
        # Replace path separators with underscores
        # e.g. v01/001.jpg -> v01_001.jpg
        flat_name = str(rel_path).replace(os.sep, "_")
    except ValueError:
        # Fallback if path is not relative
        flat_name = original_path.name

    # Paths for saving
    save_img_path = output_dir / flat_name
    save_txt_path = save_img_path.with_suffix(".txt")

    # Save Image with Detections
    # r.plot() returns the image as a numpy array (BGR)
    im_array = r.plot() 
    cv2.imwrite(str(save_img_path), im_array)

    # Save Labels if detections exist
    if len(r.boxes) > 0:
        with open(save_txt_path, "w") as f:
            for box in r.boxes:
                # box.cls is a tensor, get item
                cls = int(box.cls[0].item())
                # box.xywhn is a tensor, get list
                x, y, w, h = box.xywhn[0].tolist()
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

print("inference complete.")

running inference on catnip-data\data\manga...
saving flattened results to C:\Users\rifusaki\Desktop\catnip\catnip-data\results\inference

image 1/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1001.jpg: 288x640 (no detections), 356.3ms
image 2/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1002.jpg: 448x640 (no detections), 489.0ms
image 3/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1003.jpg: 448x640 (no detections), 386.9ms
image 4/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1004.jpg: 640x448 (no detections), 382.2ms
image 5/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1005.jpg: 640x448 (no detections), 368.9ms
image 6/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1006.jpg: 640x448 (no detections), 406.4ms
image 7/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1007.jpg: 640x448 (no detections), 351.9ms
image 8/2752 C:\Users\rifusa